# Lab 1

In [29]:
import numpy as np
from collections import defaultdict

In [30]:
class Component:
    def __init__(self, name : str, node1 : int, node2 : int, current_value : float, initial_value : float):
        self.name = name
        self.node1 = node1
        self.node2 = node2
        self.current_value = current_value
        self.initial_value = initial_value
    
    def __str__(self):
        return f"{self.name} is connected between node{self.node1} and node{self.node2}. Current value is {self.current_value}. Initial value is {self.initial_value}"

In [31]:
def read_file(filePath):
    file = open(filePath, 'r')
    lines = file.readlines()
    
    sim_time = float(lines[0])
    num_iter = float(lines[1])
    nodes = set()
    
    circuit_components = defaultdict(list)
    
    for line in lines[2:-1]:
        name, node1, node2, current_value, initial_value = line.strip('\n').split(' ')
        nodes.add(node1)
        nodes.add(node2)
        
        component = Component(name, int(node1[-1]), int(node2[-1]), float(current_value), float(initial_value))
        circuit_components[name].append(component)
        
    return sim_time, num_iter, circuit_components, len(nodes) - 1, len(circuit_components['Vsrc'])

In [32]:
def construct_Gmat(n, resistances):
    G = np.zeros((n, n))
    for res in resistances:
        G[res.node1 - 1][res.node1 - 1] += (1/res.current_value)
        if res.node2 != 0:
            G[res.node2 - 1][res.node2 - 1] += (1/res.current_value)
            G[res.node1 - 1][res.node2 - 1] -= (1/res.current_value)
            G[res.node2 - 1][res.node1 - 1] -= (1/res.current_value)
    return G

In [33]:
def construct_Bmat(n, m, voltage_sources):
    B = np.zeros((n, m))
    for column, voltage_source in enumerate(voltage_sources):
        if voltage_source.node1 != 0: 
            B[voltage_source.node1 - 1][column] = 1
        
        if voltage_source.node2 != 0:
            B[voltage_source.node2 - 1][column] = -1
    return B

In [34]:
def construct_Cmat(b_matrix):
    return b_matrix.transpose()

In [35]:
def construct_Dmat(m):
    return np.zeros((m, m))

In [36]:
def construct_Amat(G, B, C, D):
    G_C = np.concatenate((G, C), axis=0)
    B_D = np.concatenate((B, D), axis=0)
    return np.concatenate((G_C, B_D), axis=1)

In [37]:
def construct_Zmat(voltage_sources, current_sources, n ,m):
    i = np.zeros((n, 1))
    e = np.zeros((m, 1))
    
    for cs in current_sources:
        if cs.node1 != 0:
            i[cs.node1 -1] += cs.current_value 
        if cs.node2 != 0:
            i[cs.node2 -1] += cs.current_value 
    
    for column, voltage_source in enumerate(voltage_sources):
        e[column] = voltage_source.current_value
    
    return np.concatenate((i,e), axis=0)

In [46]:
# Read all needed data
simulation_time, num_iter, circuit_components, n, m =  read_file('./testcases/4.txt')

print(n, m)

2 1


In [48]:
# Construct G matrix
G = construct_Gmat(n, circuit_components['R'])
G

array([[ 2.5, -2. ],
       [-2. ,  2.5]])

In [49]:
# Construct B matrix
B = construct_Bmat(n, m, circuit_components['Vsrc'])
B

array([[ 1.],
       [-1.]])

In [50]:
# Construct C matrix
C = construct_Cmat(B)
C

array([[ 1., -1.]])

In [51]:
# Construct D matrix
D = construct_Dmat(m)
D

array([[0.]])

In [52]:
# Construct A matrix
A = construct_Amat(G=G, C=C, B=B, D=D)
A

array([[ 2.5, -2. ,  1. ],
       [-2. ,  2.5, -1. ],
       [ 1. , -1. ,  0. ]])

In [53]:
# Construct Z matrix
Z = construct_Zmat(circuit_components['Vsrc'], circuit_components['Isrc'], n, m)
Z

array([[1.],
       [0.],
       [2.]])

In [54]:
# Solve X matrix
inverse_A = np.linalg.inv(A)
X = np.matmul(inverse_A, Z)
X

array([[ 2.00000000e+00],
       [ 2.22044605e-16],
       [-4.00000000e+00]])

# Lab 2